In [1]:
#import relevent libraries
import pandas as pd


In [ ]:
#load data into dataframe

file = "allegations_202007271729.csv"

complaints_df = pd.read_csv(file)

complaints_df.info()

In [57]:
# Functions for EDA, cleaning and for later QA

def data_shape(dataframe):
    if dataframe.shape() != (0,0):
        return dataframe.shape()
    else:
        return "There is an error with your dataframe"

def check_dups(dataframe):
    duplicates = dataframe[dataframe.duplicated()]
    num_dups = len(duplicates)
    if num_dups > 0:
        return f"There are {num_dups} duplicates in your dataset", duplicates
    else:
        return f"There are {num_dups} duplicates in your dataset"

def drop_dups(dataframe):
    dataframe = dataframe.drop_duplicates()
    return dataframe

def check_nulls(dataframe):
    num_nulls = dataframe.isnull().sum()
    return num_nulls

def drop_nulls(dataframe):
    dataframe = dataframe.dropna()
    return dataframe

#function to combine month and year
def make_complaint_date(dataframe, year_column, month_column):
    dataframe['complaint_date'] = pd.to_datetime(dataframe[year_column].astype(str) + '-' + dataframe[month_column].astype(str) + '-01')
    return dataframe

def make_resolved_date(dataframe, year_column, month_column):
    dataframe['resolved_date'] = pd.to_datetime(dataframe[year_column].astype(str) + '-' + dataframe[month_column].astype(str) + '-01')
    return dataframe
    
#Change M and F to Male and Female respectively
def longform_sex(dataframe, gender_column):
    dataframe[gender_column] = dataframe[gender_column].replace('M', 'Male').replace('F', 'Female')
    return dataframe

#dictionaries to fix abbreviations
rank_dict = {'POM': 'Police Officer', 
        'POF': 'Police Officer', 'PO': 
        'Police Officer',
        'PSA': 'Police Officer Special Assignment',
        'SGT': 'Sergeant',
        'SSA': 'Sergeant Special Assignment',
        'SDS': 'Sergeant Detective Squad',
        'DT3': 'Detective Grade 3',
        'DT2': 'Detective Grade 2',
        'DT1': 'Detective Grade 1',
        'DTS': 'Detective Specialist',
        'LT': 'Lieutenant',
        'LSA': 'Lieutenant Special Assignment',
        'LCD': 'Lieutenanct Commander Detective',
        'CPT': 'Captain',
        'SRG': 'Police Surgeon',
        'SCS': 'Supervisory Chief Surgeon',
        'DCS': 'Deputy Chief Surgeon',
        'DI': 'Deputy Inspector',
        'INS': 'Inspector',
        'DC': 'Deputy Chief',
        'AC': 'Assistant Chief',
        'COD': 'Chief of Department',
        'CCA': 'Chief of Community Affairs',
        'CCT': 'Chief of Counterterrorism Bureau',
        'CD': 'Chief of Detectives',
        'CMS': 'Chief of Manhattan South',
        'COH': 'Chief of Housing',
        'COI': 'Chief of Intelligence',
        'COP': 'Chief of Patrol',
        'COD': 'Chieft of Department',
        'COS': 'Chief of Staff',
        'COT': 'Chief of Transportation',
        'CPB': 'Chief of Personnel Bureau'}




In [58]:
make_resolved_date(complaints_df, 'year_closed', 'month_closed')


,unique_mos_id,first_name,last_name,command_now,shield_no,complaint_id,month_received,year_received,month_closed,year_closed,...,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition,complaint_date,resolved_date
0,10004,Jonathan,Ruiz,078 PCT,8409,42835,7,2019,5,2020,...,Female,38.0,Abuse of Authority,Failure to provide RTKA card,78.0,Report-domestic dispute,No arrest made or summons issued,Substantiated (Command Lvl Instructions),2019-07-01,2020-05-01
1,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,...,Male,26.0,Discourtesy,Action,67.0,Moving violation,Moving violation summons issued,Substantiated (Charges),2011-11-01,2012-08-01
2,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,...,Male,26.0,Offensive Language,Race,67.0,Moving violation,Moving violation summons issued,Substantiated (Charges),2011-11-01,2012-08-01
3,10007,John,Sears,078 PCT,5952,26146,7,2012,9,2013,...,Male,45.0,Abuse of Authority,Question,67.0,PD suspected C/V of violation/crime - street,No arrest made or summons issued,Substantiated (Charges),2012-07-01,2013-09-01
4,10009,Noemi,Sierra,078 PCT,24058,40253,8,2018,2,2019,...,NaN,16.0,Force,Physical force,67.0,Report-dispute,Arrest - other violation/crime,Substantiated (Command Discipline A),2018-08-01,2019-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33353,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,Male,21.0,Discourtesy,Word,66.0,Moving violation,Moving violation summons issued,Unsubstantiated,2016-08-01,2017-02-01
33354,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,Male,21.0,Abuse of Authority,Interference with recording,66.0,Moving violation,Moving violation summons issued,Unsubstantiated,2016-08-01,2017-02-01
33355,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,Male,21.0,Abuse of Authority,Search (of person),66.0,Moving violation,Moving violation summons issued,Substantiated (Formalized Training),2016-08-01,2017-02-01
33356,9992,Tomasz,Pulawski,078 PCT,2642,35671,8,2016,2,2017,...,Male,21.0,Abuse of Authority,Vehicle search,66.0,Moving violation,Moving violation summons issued,Substantiated (Formalized Training),2016-08-01,2017-02-01


0          Male
1          Male
2          Male
3          Male
4        Female
          ...  
33353      Male
33354      Male
33355      Male
33356      Male
33357      Male
Name: mos_gender, Length: 33358, dtype: object

In [19]:
#drop nulls to determine how to fill in nullls
complaints_dropna = drop_nulls(complaints_df)

unique_mos_id               0
first_name                  0
last_name                   0
command_now                 0
shield_no                   0
complaint_id                0
month_received              0
year_received               0
month_closed                0
year_closed                 0
command_at_incident         0
rank_abbrev_incident        0
rank_abbrev_now             0
rank_now                    0
rank_incident               0
mos_ethnicity               0
mos_gender                  0
mos_age_incident            0
complainant_ethnicity       0
complainant_gender          0
complainant_age_incident    0
fado_type                   0
allegation                  0
precinct                    0
contact_reason              0
outcome_description         0
board_disposition           0
dtype: int64

In [34]:
complaints_over10 = complaints_dropna[complaints_dropna['complainant_age_incident'] >= 10]
complaints_over10['complainant_age_incident'].describe()

count    27996.000000
mean        32.677275
std         12.182656
min         10.000000
25%         23.000000
50%         30.000000
75%         41.000000
max        101.000000
Name: complainant_age_incident, dtype: float64

In [ ]:
#figure out what to do with nulls
#command at incidence 1544 - maybe do count of values in non-null, count $of total then assign at random for nulls
#complainant race, gener, age have most nulls and crucial data
#allegation has 1
#precent has 24
#contact_reason 199
#outcome description 56
# need to fix dates too